# Capstone Design: Sedimentation Tank Remainder

### Team SYM

### Shiyao Sun and Melanie Lim

### Monroe's comments 
(include this in your next submission so I can verify that you've addressed all of these issues.)
<font color="red">  </font>
* <font color="red">  Add an introduction explaining the problem that you are solving. The goal is to design a sed tank that can be used in many communities.
* Did you review the mathcad file that was used as the previous design? I get the impression that you didn't use that very valuable resource.</font> 
* Define the problem and explain how you will solve it.</font>
* Set this up for working in Python. Define the constants that you already know. You can look in the expert inputs file (see the mathcad source files linked below) to see if there are any other constraints you are missing for the sed tank.
* Start with the python code from the sed tank design challenge.
For the next submission you will want to have the majority of the sed tank designed. This is a significant challenge. The AIDE team can help specify what level of detail they need to be able to draw the sed tank.

It seems that you missed the guidance for this report. See https://confluence.cornell.edu/display/cee4540/DC+Capstone+Design
well documented report created in Jupyter with responses to comments received from previous submissions.
The report must be formatted to be one page wide and must be organized with text used to introduce and explain equations, images, and design methods.
Explain ALL calculations step by step including why you are doing the calculation.
Present each equation you use in a markdown cell using beautiful Latex and then follow that with the code cell with the function written in Python.
If you need to display results in a table, use a DataFrame to make a nicely formatted result that changes with input parameters.
The report must explain why you have made the decisions that you have made.
Explain the physics of the problem and how that influences your solution scheme.
Explain the constraints 
Document your sources of information with in text citations and a bibliography.
Relevant equations created using Latex and included with the text of the report.
Jupyter worksheet must be organized as a stand alone report that can be read and easily understood from beginning to end.
Include sketches, drawings, graphs as part of the Jupyter document. Either include any required images and zip all of the files together for submission.
Include sketches that show the geometry of your system and make sure that all variables are clearly defined. Write in paragraph format so that your work is easy to understand.
Clearly explain which equations you developed and which equations you borrowed from the AguaClara source code.
Spell check before submitting and proofread the entire report.

In [2]:
from aide_design.play import*
from aide_design import floc_model as floc
from pytexit import py2tex
import math

<div class="alert alert-block alert-info">

# <u>Introduction</u>

The goal of this Capstone Design challenge is to design a sedimentation tank that can be used in multiple communities. According to given inputs focused on the remainder of the sedimentation tank, the tank dimensions can be calculated using flow equations and functions we have learned throughout this semester.


<font color="red">
* Your list of constraints isn't the list that you are using for your design. Create a list that precisely matches your design steps and in python assign those constraints to variables. For example, the capture velocity is a design constraint that you use. The physical size of the sedimentation tank is NOT a constraint.
</font>


## Constraints

Material Properties:
- Amount of material
- Cost of material/construction

<br>
- Timing of floc settlement in the tank
- Upflow velocity in the floc blanket
- Capture velocity
- **Design flow rate**
- Plate settler spacing and width
- Minimum depth of the floc blanket
- Head loss through orifices in the effluent manifold

Operator/User Specifications: <font color="red"> How is this different from important constraints? </font>
- Inflow data (fluid temperature, turbidity, capture velocity)

Major Design Alternatives:
- Vertical flow sedimentation tank
- Horizontal flow sed tank


<font color="red"> 
Explain how you will choose which of these alternatives to use in your design. 

Below you define SDR more than once. Fix this.

Move ALL of the constraints to one cell and organize them by type of constraint. See if you can identify the different types of constraints. For example, one type is material properties.
</font>


<br>

In [3]:
# We are experimenting with using head loss at the exit of the diffusers as
# a reasonable design constraint for the design of the
# inlet manifold/diffuser system. This head loss in the exit of the diffuser
# will allow higher velocities (and pressure recovery) in the manifold pipe
# and thus will enable use of smaller diameter manifold pipes.
# -----
# We've set a maximum head loss in the sedimentation tank inlet:
headloss_sed_inlet_max = 1 * u.cm

# The manifold and diffuser pipes used in the sedimentation tank have an SDR of 26.
SDR=26

# Pi_jet_plane is used to estimate the maximum energy dissipation rate in a plane jet. 
# A plane jet is uniform in one dimension. A plane jet can be formed by a long narrow slot.
Pi_jet_plane = 0.225

# Design temperature will be helpful to calculate viscosity
T_design = 15*u.degC

# Plant Flowrate:
flow_plant = 60 * u.L/u.s

# Upflow velocity at the top of the floc blanket:
V_sed_up = 1 * u.mm/u.s

# The corrugated plastic sheets used to make the plate settlers:
W_sed = 42 * u.inch
thickness_sed_plate = 2 * u.mm

# The plate settlers are angled 60° from the horizontal:
angle_sed_plate = 60 * u.deg

# The plate setters are spaced 2.5cm apart (this is the perpendicular
# distance between plates, not the horizontal distance between plates):
s_sed_plate = 2.5 * u.cm

# Plate settler capture velocity:
V_sed_capture = 0.12 * u.mm/u.s

# The minimum port flow (from the first port) divided by the maximum port 
# flow (from the last port) for flow division between sedimentation tanks 
# and for flow distribution from the inlet manifold should be at least:
Pi_sed_manifold_flow = 0.8

g = pc.gravity

## Plate Settlers

Page in lecture notes on various velocities through the sed tank / plate settlers about V_up and V_vert in plate settlers
* As the length of the sed tank decreases, the effect of the small wasted area at the end of the row of plate settlers row increases because the upflow velocity in floc blanket is kept constant
* Therefore, we must increase the length of plate settlers as the length of the sed tank decreases
* Standardize all plate settlers as 2 ft long with 2.5 cm spacing
    * If these dimensions meet the capture velocity requirement, use as standard design dimensions
    * Height of plates can also be calculated

<br>

* <font color="red"> Include definition sketch showing dimensions with variable names.
* Calculate the effect of the wasted space at the end of the plate settlers.
* Show the equations that you use in Latex. Perhaps don't need all equations, but key equations should be shown.
</font>


In [4]:
# These initializations can be easily changed according to different design parameters
V_Upflow = 3 * (u.mm/u.s)
V_Capture = 0.2 * (u.mm/u.s)
Spacing_Settlers = 2.5 *(u.cm)
Angle_sed_plate = 60 * (u.deg)
Thickness_sed_plate = 0 * (u.mm)

def C_to_C_Distance(Spacing_Settlers, Thickness_sed_plate):
    # Return the center to center distance between two settler plates
    return Spacing_Settlers + Thickness_sed_plate

def Len_sed_plate(V_Upflow, V_Capture, Spacing_Settlers, Angle_sed_plate, Thickness_sed_plate):
    # Return the length of the plate settlers
    Top = Spacing_Settlers * (V_Upflow / V_Capture -1) + Thickness_sed_plate * (V_Upflow / V_Capture)
    Bottom = math.cos(Angle_sed_plate) * math.sin(Angle_sed_plate)
    Len_sed_plate = Top / Bottom
    return Len_sed_plate.to(u.cm)

def Vertical_Height_plate(V_Upflow, V_Capture, Spacing_Settlers, Angle_sed_plate, Thickness_sed_plate):
    # Return the vertical length of the plate settlers
    len_sed_plate = Len_sed_plate(V_Upflow, V_Capture, Spacing_Settlers, Angle_sed_plate, Thickness_sed_plate)
    return (len_sed_plate **2 - (Len_sed_plate * math.cos(Angle_sed_plate))**2) ** (1/2)

def Min_Adequate_Flow():
    
    return min_adeq_flow



<font color="red"> Calculate the minimum flow for a sed tank for which your standard plate settler design is adequate. Make a proposal for how to handle the design for any cases where the design capture velocity is not met. 

Test your code to make sure it works. Calculate values and print them.</font>

## Floc Blankets

We need a function for the length of floc blanket. Assume the height/depth of the floc blanket to be 25cm.
<font color="red"> What do you mean by the length of the floc blanket? How are you calculating this? Show the definition sketch that you are using to define the depth of the floc blanket 
Head loss for a floc blanket isn't an input. The operator doesn't have the ability to control the concentration of clay in the floc blanket. We don't have design equations for the floc blanket yet.
</font>

In [8]:
# These initializations can be easily changed according to different design parameters
Con_Clay = 3 * (u.g/u.L)
V_Upflow = 1 * (u.mm/u.s)
Temp_Water = 20 * (u.degC)
Den_Clay = 2650 * (u.kg/(u.m**3))
Porosity_FB = 1
Den_Water = pc.density_water(Temp_Water)
nu = pc.viscosity_kinematic(Temp_Water)
headloss_Floc_Blanket = 0.2 * (u.cm)

def Depth_Floc_Blanket(headloss_Floc_Blanket,Den_Water,Den_Clay,Con_Clay):
    # Return the depth of the floc blanket
    RHS = (1/Den_Water - 1/Den_Clay) * Con_Clay
    return (headloss_Floc_Blanket / RHS)




## Effluent Manifolds

## Inlet Manifolds

Assume manifold pipe is thin-walled with a SDR of 41.
* ratio between smallest and largest velocity for diffusers
* maximum head loss for jet coming out of diffuser, used 
* these ^ can find diameter for pipe to be and design for diffusers


In [6]:
Manifold_SDR = 41


## Inlet Diffusers

We need a function to calculate the length of diffuser.
<font color="red"> I'm not sure what constraint sets this length. Shorter is stronger. Longer does a better job of creating a uniform velocity from the diffuser tip, but this likely isn't very significant. For now I'd suggest using a length of 6 inches.

You have equations below, but you don't use them for anything. This is true throughout the document. Why aren't there any calculations or design results? You have a design challenge where you designed inlet diffusers. Why not use that as a starting point?
 </font>

In [7]:
# These initializations can be easily changed according to different design parameters
Diffuser_SDR=26
ND_sed_diffuser = 1 * u.inch
ID_sed_diffuser = pipe.ID_SDR(ND_sed_diffuser,SDR)
OD_sed_diffuser = pipe.OD(ND_sed_diffuser)

def V_Jet(headloss_jet):
    return (2* pc.gravity * headloss_jet) ** (1/2)

def Width_Diffuser(V_sed,W_sed,B_Diffuser,headloss_jet,S_Diffuser):
    return V_sed * W_sed * B_Diffuser / V_Jet(headloss_jet) / S_Diffuser



## Jet Reverser

There is no design needed for jet reverser. Take 3" PVC pipe cut in half. Assume the pipe has a SDR of 26.

## Floc Hopper and floc weir

* certain distance Height beneath plate settlers before you get to floc weir
* idea: clear water in this Height (H_clearwater) ~10cm ish


<font color="red"> I believe you missed the design of the effluent manifold. Then calculate the entire depth of the sedimentation tank. </font>